## クーロン相互作用のパラメータを変えシミュレーションを流すノートブック

In [ ]:
#path
using CUDA
using EzXML
using MDToolbox
using Statistics

pdb_filepath = "./alanine-dipeptide-nowater.pdb" #pdbのパス
top = readpdb(pdb_filepath)
ff_start_filepath  = "./data/amber14/protein.ff14SB.xml" #startのffのパス
out_dir = "sim_qoulomb" #出力するディレクトリ
isdir(out_dir) || mkdir(out_dir)

#setting
njobs = 10 #並列して流すシミュレーションの数
nsteps = 2_500_000_000 #シミュレーションのステップ数
gpu_id = "n4"
update_ratio = 0.5 #forcefieldをどの程度変えるか

## シミュレーション

In [ ]:
# 一つのシミュレーションを流すジョブスクリプト
# ff_filepath　　　  :xml formatの力場ファイルのパス
# out_traj_dir      :アウトプットするディレクトリのパス
# i                 :流すシミュレーションの通し番号

function run_job(ff_filepath, traj_filepath, log_filepath, i)
    sbatch_file = "temp_$(i).sh"
    
    open(sbatch_file, "w") do of
        println(of, "#!/bin/bash")
        println(of, "#SBATCH -p all")
        println(of, "#SBATCH -J sim$(i) # job name")
        println(of, "#SBATCH -n 1  # num of total mpi processes")
        println(of, "#SBATCH -c 1  # num of threads per mpi processes")
        println(of, "#SBATCH -o $(log_filepath)")
        println(of, "export CUDA_VISIBLE_DEVICES=$(i)")
        #println(of, "python sim.py $(pdb_filepath) $(ff_filepath) $(traj_filepath) $(nsteps)")
        println(of, "time python sim.py $(pdb_filepath) $(ff_filepath) $(traj_filepath) $(nsteps)")
    end
    sleep(5)
    run(`sbatch --gpus-per-node=1 -w $(gpu_id) $(sbatch_file)`)
    #run(`sbatch -w $(gpu_id) $(sbatch_file)`)
    sleep(5)
    rm(sbatch_file)
end

In [ ]:
resname_list = top.resname
atomname_list = top.atomname

## ffを作成する関数

In [ ]:
struct Atom
    name::String      # 原子の名前
    charge::Float64   # 原子の電荷
end

struct Residue
    name::String             # 残基の名前
    atoms::Vector{Atom}      # 残基内の全原子のリスト
end

In [ ]:
function parse_residue(residue_node)
    resname = [nodecontent(i) for i in eachattribute(residue_node)][1]
    atoms = Vector{Atom}()
    
    # 残基内の各原子を取得
    for atom_node in eachelement(residue_node)
        node_name = [nodename(i) for i in eachattribute(atom_node)]
        node_content = [nodecontent(i) for i in eachattribute(atom_node)]
        if length(node_content[node_name .== "name"]) == 0
            continue
        end
        q = parse(Float32, node_content[node_name .== "charge"][1])
        atomname = node_content[node_name .== "name"][1]
        
        push!(atoms, Atom(atomname, q))
    end
    
    # Residue構造体を作成
    return Residue(resname, atoms)
end

In [ ]:
function find_matching_atoms(residue::Residue, atom_names::Vector{String})
    # 残基内に指定された原子名のいずれかが含まれている場合、それらのリストを返す
    matching_atoms = [atom.name for atom in residue.atoms if atom.name in atom_names]
    return matching_atoms
end

In [ ]:
function create_ff_qoulomb(ff_input_filepath, ff_output_filepath, update_ratio)
    input_xml = readxml(ff_input_filepath)
    output_xml = deepcopy(input_xml)
    xmlroot = root(output_xml)
    children = elements(xmlroot)
    children_name = nodename.(children)
    residues_indes = children_name .== "Residues"
    residues = children[residues_indes][1]
    
    resname_list = top.resname
    atomname_list = top.atomname
    
    q_ALA_HB = 0.0603 * (1 + update_ratio * (2 * rand() - 1))
    q_ACE_HH = 0.1123 * (1 + update_ratio * (2 * rand() - 1))
    q_NME_HH = 0.0976 * (1 + update_ratio * (2 * rand() - 1))
    
    for residue in eachelement(residues)
        resname = [nodecontent(i) for i in eachattribute(residue)][1]
        if resname in resname_list

            tmp = []
            for element_residue in eachelement(residue)
                #println([nodename(i) for i in eachattribute(element_residue)])
                #println([nodecontent(i) for i in eachattribute(element_residue)])
                node_name = [nodename(i) for i in eachattribute(element_residue)]
                node_content = [nodecontent(i) for i in eachattribute(element_residue)]
                if length(node_content[node_name .== "name"]) == 0
                    continue
                end
                q = parse(Float32, node_content[node_name .== "charge"][1])
                atomname = node_content[node_name .== "name"][1]
                #println("atomname = $atomname, q = $q")
                
                q_updated = q * (1 + update_ratio*(2 * rand() - 1))
    
                if(resname == "ALA" && occursin(r"^HB.*", atomname))
                    q_updated = q_ALA_HB
                end
                if(resname == "NME" && occursin(r"^HH.*", atomname))
                    q_updated = q_NME_HH
                end
                if(resname == "ACE" && occursin(r"^HH.*", atomname))
                    q_updated = q_ACE_HH
                end
                push!(tmp, q_updated)
                element_residue["charge"] = string(q_updated)
            end
            for element_residue in eachelement(residue)
                #println([nodename(i) for i in eachattribute(element_residue)])
                #println([nodecontent(i) for i in eachattribute(element_residue)])
                node_name = [nodename(i) for i in eachattribute(element_residue)]
                node_content = [nodecontent(i) for i in eachattribute(element_residue)]
                if length(node_content[node_name .== "name"]) == 0
                    continue
                end
                q = parse(Float32, node_content[node_name .== "charge"][1])
                atomname = node_content[node_name .== "name"][1]
                #println("atomname = $atomname, q = $q")
                
                q_updated = q - mean(tmp)
                element_residue["charge"] = string(q_updated)
            end
        end
    end
    write(ff_output_filepath, output_xml)
end

In [ ]:
function input_ff_qoulomb(ff_filepath)
    charge = zeros(Float64, length(atomname_list))
    xml = readxml(ff_filepath)
    xmlroot = root(xml)
    children = elements(xmlroot)
    children_name = nodename.(children)
    residues_indes = children_name .== "Residues"
    residues = children[residues_indes][1]
    
    for residue in eachelement(residues)
        resname = [nodecontent(i) for i in eachattribute(residue)][1]
        if resname in resname_list
            atoms = atomname_list[resname_list .== resname]
            for element_residue in eachelement(residue)   
                node_name = [nodename(i) for i in eachattribute(element_residue)]
                node_content = [nodecontent(i) for i in eachattribute(element_residue)]
    
                if length(node_content[node_name .== "name"]) == 0
                    continue
                end
                atomname = node_content[node_name .== "name"][1]
                if atomname in atoms
                    q = node_content[node_name .== "charge"][1]
                    #println(atomname_list .== atomname .&& resname_list .== resname)  
                    charge[atomname_list .== atomname .&& resname_list .== resname] .= parse(Float64, q)
                end               
            end
        end
    end
    return charge
end

In [ ]:
for i in 1:njobs
    sim_dir = joinpath(out_dir, "sim_$(i)")
    isdir(sim_dir) || mkdir(sim_dir)

    ff_filepath = joinpath(sim_dir, "sim_$(i).xml")
    create_ff_qoulomb(ff_start_filepath, ff_filepath, update_ratio)

    traj_filepath = joinpath(sim_dir, "traj_$(i)")
    log_filepath = joinpath(sim_dir, "sim_$(i).log")

    run_job(ff_filepath, traj_filepath, log_filepath, i)
end